In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

## Code taken from experiments:

In [3]:
# Load lastfm data
baseline = pd.read_csv('../results/lastfm/agent_topk=10-12-1/baseline_avg.csv') # PGPR
LIRopt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/LIRopt_moving_alpha_avg.csv') # R-PGPR
SEPopt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/SEPopt_moving_alpha_avg.csv') # P-PGPR 
ETDopt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/ETDopt_moving_alpha_avg.csv') # D-PGPR
ETD_SEP_opt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/ETD_SEP_opt_moving_alpha_avg.csv') # DP-PGPR
SEP_LIR_opt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/SEP_LIR_opt_moving_alpha_avg.csv') # PR-PGPR
ETD_LIR_opt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/ETD_LIR_opt_moving_alpha_avg.csv') # DR-PGPR
ETD_SEP_LIR_opt = pd.read_csv('../results/lastfm/agent_topk=10-12-1/ETD_SEP_LIR_opt_moving_alpha_avg.csv') # DPR-PGPR

# Loas ml1m data
baseline_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/baseline_avg.csv') # PGPR
LIRopt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/LIRopt_moving_alpha_avg.csv') # R-PGPR
SEPopt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/SEPopt_moving_alpha_avg.csv') # P-PGPR
ETDopt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/ETDopt_moving_alpha_avg.csv') # D-PGPR
ETD_SEP_opt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/ETD_SEP_opt_moving_alpha_avg.csv') # DP-PGPR
SEP_LIR_opt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/SEP_LIR_opt_moving_alpha_avg.csv') # PR-PGPR
ETD_LIR_opt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/ETD_LIR_opt_moving_alpha_avg.csv') # DR-PGPR
ETD_SEP_LIR_opt_ml1m = pd.read_csv('../results/ml1m/agent_topk=10-12-1/ETD_SEP_LIR_opt_moving_alpha_avg.csv') # DPR-PGPR



In [ ]:
alpha_values = [0.1, 0.2, 0.3, 0.4, 0.5]
metrics = ['ndcg', 'LIR', 'SEP', 'ETD']
group = 'Overall'

# Combined data
combined_df_lastfm = pd.concat([baseline, LIRopt, SEPopt, ETDopt, ETD_SEP_opt, SEP_LIR_opt, ETD_LIR_opt, ETD_SEP_LIR_opt])
combined_df_ml1m = pd.concat([baseline_ml1m, LIRopt_ml1m, SEPopt_ml1m, ETDopt_ml1m, ETD_SEP_opt_ml1m, SEP_LIR_opt_ml1m, ETD_LIR_opt_ml1m, ETD_SEP_LIR_opt_ml1m])

def compare_to_baseline(combined_df):
    # Filter by alpha and group by Overall and metric ndcg
    filtered_df = combined_df[
        (combined_df['alpha'].isin(alpha_values)) & 
        (combined_df['metric'].isin(metrics)) &
        (combined_df['group'] == group)
    ]

    baseline_df = filtered_df[filtered_df['opt'] == 'baseline']
    optimization_df = filtered_df[filtered_df['opt'] != 'baseline']


    merged_df = optimization_df.merge(
        baseline_df,
        on=['alpha', 'metric', 'group'],
        suffixes=('_opt', '_baseline')
    )

    merged_df = merged_df.sort_values(by=['metric', 'alpha'])
    merged_df['opt_opt'] = merged_df['opt_opt'].str.replace('LIRopt', 'R-PGPR')
    merged_df['opt_opt'] = merged_df['opt_opt'].str.replace('SEPopt', 'P-PGPR')
    merged_df['opt_opt'] = merged_df['opt_opt'].str.replace('ETDopt', 'D-PGPR')
    
    return merged_df

def get_pvalue(merged_df, metric, opt):
    opt_df = merged_df[(merged_df['metric'] == metric)]
    opt_df = opt_df[(opt_df['opt_opt'] == opt)]
    opt_df = opt_df.loc[:, ['data_opt', 'data_baseline']]
    (statistic, pvalue) = stats.kruskal(opt_df['data_opt'], opt_df['data_baseline'])
    return pvalue

def significance(pvalue):
    return pvalue<0.05

merged_df_lastfm = compare_to_baseline(combined_df_lastfm)
merged_df_ml1m = compare_to_baseline(combined_df_ml1m)

pvalue = get_pvalue(merged_df_lastfm, 'LIR', 'P-PGPR')
print(pvalue)
significance(pvalue)



0.005345676872654257


True